In [4]:
from PIL import Image
import struct
import ctypes

img = Image.open('test.png')
(w, h) = img.size[0:2]
pix = img.load()
buff = ctypes.create_string_buffer(4 * w * h)
offset = 0
for j in range(h):
	for i in range(w):
		r = bytes((pix[i, j][0],))
		g = bytes((pix[i, j][1],))
		b = bytes((pix[i, j][2],))
		a = bytes((255,))
		struct.pack_into('cccc', buff, offset, r, g, b, a)
		offset += 4
out = open('test.data', 'wb')
out.write(struct.pack('ii', w, h))
out.write(buff.raw)
out.close()

In [12]:
%%writefile image.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>

#define MAX_CLASSES 32

#define CSC(call)  									                \
do {											                    \
	cudaError_t res = call;							                \
	if (res != cudaSuccess) {							            \
		fprintf(stderr, "ERROR in %s:%d. Message: %s\n",			\
				__FILE__, __LINE__, cudaGetErrorString(res));		\
		exit(0);								                    \
	}										                        \
} while(0)

__constant__ float c_avg[MAX_CLASSES][3]; // усредненные векторы классов
__constant__ int c_nc; // количество классов

typedef unsigned char uchar;

typedef struct {
    uchar x, y, z, w;
} uchar4_custom;

int loadImage(const char *fname, int *pw, int *ph, uchar4_custom **pdata) {
    FILE *f = fopen(fname, "rb");
    if (!f) {
        fprintf(stderr, "Не удалось открыть файл %s\n", fname);
        return 0;
    }

    unsigned int w = 0, h = 0;
    if (fread(&w, sizeof(unsigned int), 1, f) != 1 || fread(&h, sizeof(unsigned int), 1, f) != 1) {
        fprintf(stderr, "Ошибка чтения заголовка\n");
        fclose(f);
        return 0;
    }

    size_t count = (size_t)w * (size_t)h;
    uchar4_custom *buf = (uchar4_custom *)malloc(count * sizeof(uchar4_custom));
    if (!buf) {
        fprintf(stderr, "Ошибка выделения памяти\n");
        fclose(f);
        return 0;
    }

    size_t got = fread(buf, sizeof(uchar4_custom), count, f);
    fclose(f);
    if (got != count) {
        fprintf(stderr, "Ошибка чтения данных изображения\n");
        free(buf);
        return 0;
    }

    *pw = (int)w;
    *ph = (int)h;
    *pdata = buf;
    return 1;
}

int saveImage(const char *fname, int w, int h, const uchar4_custom *data) {
    FILE *f = fopen(fname, "wb");
    if (!f) {
        fprintf(stderr, "Не удалось создать файл %s\n", fname);
        return 0;
    }

    unsigned int ww = (unsigned int)w;
    unsigned int hh = (unsigned int)h;
    fwrite(&ww, sizeof(unsigned int), 1, f);
    fwrite(&hh, sizeof(unsigned int), 1, f);
    fwrite(data, sizeof(uchar4_custom), (size_t)w * (size_t)h, f);
    fclose(f);
    return 1;
}

__global__ void kernel(uchar4_custom *input, uchar4_custom *output, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= size) {
        return;
    }

    float r = input[idx].x;
    float g = input[idx].y;
    float b = input[idx].z;

    float len_p = sqrtf(r * r + g * g + b * b);
    if (len_p < 1e-6f) {
        len_p = 1.0f;
    }

    float max_score = -1e9f;
    int best_class = 0;

    for (int j = 0; j < c_nc; ++j) {
        float dot = r * c_avg[j][0] + g * c_avg[j][1] + b * c_avg[j][2];
        float score = dot / len_p;
        if (score > max_score) {
            max_score = score;
            best_class = j;
        }
    }

    output[idx] = input[idx];
    output[idx].w = (uchar)best_class;
}

int main() {
    char inPath[256], outPath[256];

    if (!fgets(inPath, sizeof(inPath), stdin)) {
        fprintf(stderr, "Ошибка чтения пути входного файла\n");
        return 1;
    }
    if (!fgets(outPath, sizeof(outPath), stdin)) {
        fprintf(stderr, "Ошибка чтения пути выходного файла\n");
        return 1;
    }

    inPath[strcspn(inPath, "\r\n")] = '\0';
    outPath[strcspn(outPath, "\r\n")] = '\0';

    if (strlen(inPath) == 0 || strlen(outPath) == 0) {
        fprintf(stderr, "Пути не должны быть пустыми\n");
        return 1;
    }

    int w, h;
    uchar4_custom *data = NULL;
    if (!loadImage(inPath, &w, &h, &data)) {
        fprintf(stderr, "Ошибка загрузки входного файла: %s\n", inPath);
        return 1;
    }

    int nc;
    scanf("%d", &nc);
    if (nc > MAX_CLASSES) {
        fprintf(stderr, "Слишком много классов (максимум %d)\n", MAX_CLASSES);
        return 1;
    }

    float avg[MAX_CLASSES][3] = {0.0f};
    int np;
    for (int j = 0; j < nc; ++j) {
        scanf("%d", &np);
        if (np <= 0) {
            fprintf(stderr, "Ошибка: класс %d не содержит пикселей\n", j);
            return 1;
        }

        for (int k = 0; k < np; ++k) {
            int x, y;
            scanf("%d %d", &x, &y);
            if (x < 0 || x >= w || y < 0 || y >= h) {
                continue;
            }
            uchar4_custom p = data[y * w + x];
            avg[j][0] += p.x;
            avg[j][1] += p.y;
            avg[j][2] += p.z;
        }

        avg[j][0] /= np;
        avg[j][1] /= np;
        avg[j][2] /= np;

        float len = sqrtf(avg[j][0]*avg[j][0] + avg[j][1]*avg[j][1] + avg[j][2]*avg[j][2]);
        if (len < 1e-6f) len = 1.0f;
        avg[j][0] /= len;
        avg[j][1] /= len;
        avg[j][2] /= len;
    }

    CSC(cudaMemcpyToSymbol(c_avg, avg, nc * 3 * sizeof(float)));
    CSC(cudaMemcpyToSymbol(c_nc, &nc, sizeof(int)));

    size_t size = (size_t)w * (size_t)h;
    uchar4_custom *dev_in = NULL, *dev_out = NULL;
    CSC(cudaMalloc(&dev_in, size * sizeof(uchar4_custom)));
    CSC(cudaMalloc(&dev_out, size * sizeof(uchar4_custom)));
    CSC(cudaMemcpy(dev_in, data, size * sizeof(uchar4_custom), cudaMemcpyHostToDevice));

    kernel<<<400000, 1024>>>(dev_in, dev_out, (int)size);
    CSC(cudaGetLastError());
    CSC(cudaMemcpy(data, dev_out, size * sizeof(uchar4_custom), cudaMemcpyDeviceToHost));

    if (!saveImage(outPath, w, h, data)) {
        fprintf(stderr, "Ошибка записи результата\n");
        free(data);
        return 1;
    }

    CSC(cudaFree(dev_in));
    CSC(cudaFree(dev_out));
    free(data);
    return 0;
}

Overwriting image.cu


In [22]:
%%shell
nvcc -arch=sm_75 image.cu
./a.out

/content/test.data
/content/test_out.data
10
10 837 140 766 225 1193 251 332 46 765 10 911 107 1152 61 1181 153 813 297 1142 4  10 590 405 458 359 690 156 968 286 359 526 223 603 433 185 1078 374 463 196 500 438  10 797 482 67 661 872 581 695 663 176 573 3 636 897 656 254 654 1173 556 769 649  10 1046 618 894 458 682 552 155 477 1001 654 531 597 955 454 932 408 955 509 667 565  10 919 310 403 378 709 254 552 3 599 278 1176 288 347 456 724 447 674 369 702 133  10 87 385 346 436 348 410 77 329 101 325 163 447 338 439 83 341 330 390 127 321  10 299 357 228 417 859 527 817 410 540 541 456 112 636 496 755 596 876 526 562 529  10 505 611 613 632 621 637 511 609 953 443 954 582 981 667 873 322 299 424 340 584  10 212 233 63 139 187 44 133 177 102 44 310 209 231 183 54 6 225 281 207 65  10 1077 550 47 409 986 383 1050 582 1149 472 102 511 35 544 969 385 238 506 1110 623


In [21]:
import numpy as np
from sklearn.cluster import KMeans
from PIL import Image
import random

# Путь к изображению
input_image_path = "/content/test.png"

# Число классов
nc = 10

# Сколько пикселей отбирать для каждой выборки
pixels_per_class = 50

# Загружаем изображение
img = Image.open(input_image_path).convert("RGB")
w, h = img.size
data = np.array(img)

# Преобразуем в список точек: [x, y, r, g, b]
points = []
for y in range(h):
    for x in range(w):
        r, g, b = data[y, x]
        points.append([r, g, b, x, y])
points = np.array(points)

# K-means по цвету
kmeans = KMeans(n_clusters=nc, random_state=42)
labels = kmeans.fit_predict(points[:, :3])

# Формируем выборки для классов
for class_idx in range(nc):
    idxs = np.where(labels == class_idx)[0]
    selected = random.sample(list(idxs), min(pixels_per_class, len(idxs)))
    print(len(selected), end=' ')
    for idx in selected:
        x, y = points[idx, 3], points[idx, 4]
        print(x, y, end=' ')
    print()

10 837 140 766 225 1193 251 332 46 765 10 911 107 1152 61 1181 153 813 297 1142 4 
10 590 405 458 359 690 156 968 286 359 526 223 603 433 185 1078 374 463 196 500 438 
10 797 482 67 661 872 581 695 663 176 573 3 636 897 656 254 654 1173 556 769 649 
10 1046 618 894 458 682 552 155 477 1001 654 531 597 955 454 932 408 955 509 667 565 
10 919 310 403 378 709 254 552 3 599 278 1176 288 347 456 724 447 674 369 702 133 
10 87 385 346 436 348 410 77 329 101 325 163 447 338 439 83 341 330 390 127 321 
10 299 357 228 417 859 527 817 410 540 541 456 112 636 496 755 596 876 526 562 529 
10 505 611 613 632 621 637 511 609 953 443 954 582 981 667 873 322 299 424 340 584 
10 212 233 63 139 187 44 133 177 102 44 310 209 231 183 54 6 225 281 207 65 
10 1077 550 47 409 986 383 1050 582 1149 472 102 511 35 544 969 385 238 506 1110 623 


In [23]:
#!/usr/bin/env python3
from PIL import Image
import struct
from random import seed, randint

# Конвертация бинарного .data обратно в PNG
# Если colorize_classes=True, то классы (alpha-канал) раскрашиваются случайными цветами

def data_to_png(input_data="test_out.data", output_png="test_out.png", colorize_classes=True):
    # Читаем бинарный файл
    with open(input_data, "rb") as f:
        w, h = struct.unpack("<ii", f.read(8))
        pixels = f.read()

    expected = w * h * 4
    if len(pixels) != expected:
        raise ValueError(f"Файл повреждён: ожидалось {expected} байт, а получено {len(pixels)}")

    if not colorize_classes:
        # просто восстанавливаем RGBA
        img = Image.frombytes("RGBA", (w, h), pixels)
    else:
        # раскрашиваем по классам (альфа = id класса)
        seed(42)
        colors = [(randint(0,255), randint(0,255), randint(0,255)) for _ in range(256)]
        out_bytes = bytearray()
        for i in range(0, len(pixels), 4):
            r = pixels[i]
            g = pixels[i+1]
            b = pixels[i+2]
            a = pixels[i+3]  # класс
            cr, cg, cb = colors[a % 256]
            out_bytes.extend((cr, cg, cb, 255))
        img = Image.frombytes("RGBA", (w, h), bytes(out_bytes))

    img.save(output_png)
    print(f"✅ DATA → PNG: '{input_data}' → '{output_png}' ({w}x{h})")


# Запуск в Colab / вручную
if __name__ == "__main__":
    data_to_png("test_out.data", "test_out.png", colorize_classes=True)

✅ DATA → PNG: 'test_out.data' → 'out_norm10.png' (1200x670)
